# Introducción
OpenStack está basado en diferentes componentes, cada uno ofreciendo una funcionalidad diferente. Aunque todos están relacionados entre sí, es necesario utilizar clientes diferentes para cada uno de ellos. Los componentes fundamentales son los siguientes:

* OpenStack Dashboard (Horizon): Interfaz web.
* OpenStack Identity (Keystone): Gestiona las identidades de los usuarios.
* OpenStack Compute (Nova): Gestiona los servidores (instancias): creación, parada, borrado, reinicio, etc. Podemos indicarle que tipo de instancia queremos a través de "flavors" (configuraciones de hardware predefinidas).
* OpenStack Image (Glance): Gestiona las imágenes de disco en las que se basarán los servidores que arranquemos.
* OpenStack Volume (Cinder): Gestiona volúmenes (disco) que podemos agregar a un servidor. Estos volúmenes persisten hasta que los borremos explícitamente, por lo que podemos cambiar el servidor al que están agregados.ç
* OpenStack Networking (Neutron): Gestiona la red de nuestras máquinas. Permite definir topologías complejas.
* OpenStack Storage (Swift): Permite almacenar ficheros (object/blob) (similar a S3).

Los clientes de Python para utilizar para cada uno de ellos son:

* `python-openstackclient`: Cliente de línea de comandos general de OpenStack, agrega todos los clientes individuales.
* `python-keystoneclient`: Librería de Python para interactuar con Keystone.
* `python-novaclient`: Librería de Python para interactuar con Nova.
* `python-glanceclient`: Librería de Python para interactuar con Glance.
* `python-cinderclient`: Librería de Python para interactuar con Cinder.
* `python-neutronclient`: Librería de Python para interactuar con Neutron.
* `python-swiftclient`: Librería de Python para interactuar con Swift.

# Parámetros globales

In [1]:
# Es necesario definir el usuario, contraseña y proyecto
# al que queremos acceder
USERNAME = "masterXX"
PASSWORD = ""
PROJECT_ID = "d2828a5b2eda4ad6a6de5cf3ef41df64"

# Parámetros de la conexión y de la versión de la API a utilizar
AUTH_URL = "https://keystone.ifca.es:5000/v3"
NOVA_VERSION = "2"
DOMAIN = "Default"

# Módulos necesarios

In [2]:
from keystoneauth1 import loading
from keystoneauth1 import session
from glanceclient import Client as g_client
from neutronclient.v2_0 import client as net_client
from novaclient import client as n_client

# Authenticación
Vamos a utilizar un objeto de sesión que se va a encargar de gestionar la autenticación (por ejemplo gestionando por nosotros la caducidad de las credenciales) en todos los clientes y en las diferentes llamadas a los mismos.

In [3]:
# Indicamos que queremos utilizar el plugin de "password", ya que
# vamos a utilizar un usuario y contraseña para conectar
loader = loading.get_plugin_loader('password')
auth = loader.load_from_options(auth_url=AUTH_URL,
                                username=USERNAME,
                                password=PASSWORD,
                                project_id=PROJECT_ID,
                                user_domain_name=DOMAIN)

# Creamos un objeto de sesión, para utilizar con todos los clientes
# que vamos a utilizar y no tener que autenticarnos cada vez con 
# cada uno de ellos
sess = session.Session(auth=auth)

# Nova

Nova es el componente que se encarga de manejar los servidores (instancias). A él le daremos las ordenes de creación, parada, etc.

Primero creamos el cliente que vamos a utilizar en el resto de la sesión.

In [4]:
nova = n_client.Client(NOVA_VERSION, session=sess)

Vamos a ver si hay algo ejecutándose ya:

In [5]:
servers = nova.servers.list()

In [6]:
for server in servers:
    print("-" * 90)
    print("Server ID:", server.id)
    print("Server name:", server.name)

Vamos a ver todos los flavors a los que tenemos acceso

In [7]:
flavors = nova.flavors.list(sort_key="name")
for flavor in flavors:
    print("-" * 90)
    print("Flavor ID:", flavor.id)
    print("Flavor:", flavor.name)
    print("RAM:", flavor.ram)
    print("CPUS:", flavor.vcpus)

------------------------------------------------------------------------------------------
Flavor ID: 786e7ddc-e0a5-4c6a-b513-795bd1b5c54f
Flavor: cm4.2xlarge
RAM: 16000
CPUS: 8
------------------------------------------------------------------------------------------
Flavor ID: 9beb31e1-0796-4a2e-9d48-8b5f41a44d89
Flavor: cm4.4xlarge
RAM: 32000
CPUS: 16
------------------------------------------------------------------------------------------
Flavor ID: c765383f-b144-4e90-8819-0cd00cabff67
Flavor: cm4.8xlarge
RAM: 64000
CPUS: 32
------------------------------------------------------------------------------------------
Flavor ID: cd775e27-af09-407b-ab34-fb999ba418bd
Flavor: cm4.large
RAM: 4000
CPUS: 2
------------------------------------------------------------------------------------------
Flavor ID: 3e880029-31a2-418a-8c69-f3330aaa3e96
Flavor: cm4.xlarge
RAM: 8000
CPUS: 4
------------------------------------------------------------------------------------------
Flavor ID: 47dcdee9-84

Podemos filtrar por RAM disponible

In [8]:
flavors_16GB = nova.flavors.list(sort_key="name", min_ram=16000)
for flavor in flavors_16GB:
    print("-" * 90)
    print("Flavor ID:", flavor.id)
    print("Flavor:", flavor.name)
    print("RAM:", flavor.ram)
    print("CPUS:", flavor.vcpus)

------------------------------------------------------------------------------------------
Flavor ID: 786e7ddc-e0a5-4c6a-b513-795bd1b5c54f
Flavor: cm4.2xlarge
RAM: 16000
CPUS: 8
------------------------------------------------------------------------------------------
Flavor ID: 9beb31e1-0796-4a2e-9d48-8b5f41a44d89
Flavor: cm4.4xlarge
RAM: 32000
CPUS: 16
------------------------------------------------------------------------------------------
Flavor ID: c765383f-b144-4e90-8819-0cd00cabff67
Flavor: cm4.8xlarge
RAM: 64000
CPUS: 32
------------------------------------------------------------------------------------------
Flavor ID: 47dcdee9-84da-4e43-baa7-bb67b3a53ed4
Flavor: g3.large
RAM: 22000
CPUS: 12
------------------------------------------------------------------------------------------
Flavor ID: d9c6ddc6-9854-401e-8f45-062bfa738d19
Flavor: m2.12xlarge
RAM: 220000
CPUS: 48
------------------------------------------------------------------------------------------
Flavor ID: 646642

Vamos a utilizar la function `to_dataframe` para mostrar los resultados de una forma más clara en el notebook, guardándolos en un DataFrame de pandas. Esto realmente NO es necesario, pero así podemos ver los resultados de una forma más sencilla y clara.

In [9]:
import pandas

def to_dataframe(objects):
    df = pandas.DataFrame()

    for obj in objects:
        if hasattr(obj, "to_dict"):
            df = df.append(obj.to_dict(), ignore_index=True)
        else:
            df = df.append(obj, ignore_index=True)

    df = df.set_index("id")
    return df

In [10]:
to_dataframe(flavors)

,OS-FLV-DISABLED:disabled,OS-FLV-EXT-DATA:ephemeral,disk,links,name,os-flavor-access:is_public,ram,rxtx_factor,swap,vcpus
id,,,,,,,,,,
786e7ddc-e0a5-4c6a-b513-795bd1b5c54f,0.0,60.0,30.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,cm4.2xlarge,1.0,16000.0,1.0,,8.0
9beb31e1-0796-4a2e-9d48-8b5f41a44d89,0.0,160.0,30.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,cm4.4xlarge,1.0,32000.0,1.0,,16.0
c765383f-b144-4e90-8819-0cd00cabff67,0.0,160.0,30.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,cm4.8xlarge,1.0,64000.0,1.0,,32.0
cd775e27-af09-407b-ab34-fb999ba418bd,0.0,30.0,30.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,cm4.large,1.0,4000.0,1.0,,2.0
3e880029-31a2-418a-8c69-f3330aaa3e96,0.0,60.0,30.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,cm4.xlarge,1.0,8000.0,1.0,,4.0
47dcdee9-84da-4e43-baa7-bb67b3a53ed4,0.0,150.0,60.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,g3.large,1.0,22000.0,1.0,,12.0
4,0.0,40.0,30.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,m1.large,1.0,7000.0,1.0,,4.0
3,0.0,40.0,10.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,m1.medium,1.0,4000.0,1.0,,2.0
2,0.0,20.0,10.0,[{'href': 'https://cloud.ifca.es:8774/v2.1/d28...,m1.small,1.0,2000.0,1.0,,1.0


Vamos a utilizar un flavor medio, por ejemplo `cm4.large` con id `cd775e27-af09-407b-ab34-fb999ba418bd`.

In [12]:
FLAVOR_ID = "cd775e27-af09-407b-ab34-fb999ba418bd"

# Selección de la imagen
Para seleccionar en que imagen queremos basar nuestra instancia tenemos que interactuar con Glance. Para ello, creamos un objeto cliente que utilizaremos para el resto de llamadas.

In [13]:
glance = g_client('2', session=sess)

In [14]:
images = glance.images.list()

In [15]:
# Vamos a listar las imágenes.
for image in images:
    print("-" * 90)
    print("Image ID", image.id)
    print("Image Name", image.name)
    break

------------------------------------------------------------------------------------------
Image ID 6a89a3bc-3389-422f-b52b-9249bc1c18cc
Image Name IFCA Ubuntu 14.04 [2017-11-30]


In [16]:
to_dataframe(images)

,architecture,checksum,container_format,created_at,disk_format,distribution,file,ifca,imgsync.sha256,min_disk,...,version,virtual_size,visibility,imgsync.sha512,os_command_line,vmcatcher_event_ad_mpuri,app_01,app_01_ver,distro,distro_ver
id,,,,,,,,,,,,,,,,,,,,,
288d0d6a-17e2-4f16-a230-aa288dd146e1,x86_64,921539f1ebcaf8e822e777444e6f5090,bare,2017-11-30T04:00:27Z,qcow2,ubuntu,/v2/images/288d0d6a-17e2-4f16-a230-aa288dd146e...,true,0c6ac917eb58c524d72d30d694f7b21ea6e09b696fbb72...,0.0,...,16.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN
d23fbac1-247d-4b16-ae6d-a0734f6122d3,x86_64,fda3aa20b1684b00b1be9ef789fe5fd3,bare,2017-11-29T04:13:41Z,qcow2,ubuntu,/v2/images/d23fbac1-247d-4b16-ae6d-a0734f6122d...,true,6b05a803ab8c2852e6a524c4b2b95e035cf0321c40dc35...,0.0,...,14.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0df398e8-5353-46ee-ada3-af16a32bd7a2,x86_64,64de8fffcfe648045d1a362d37ac8fe3,bare,2017-11-29T04:13:01Z,qcow2,centos,/v2/images/0df398e8-5353-46ee-ada3-af16a32bd7a...,true,NaN,0.0,...,6,None,public,40ce75ab89c2da20676cd2e6270c7152e6bf5f1031ff08...,NaN,NaN,NaN,NaN,NaN,NaN
4a2b369d-f3bc-4b73-b50a-cbd3687be43e,x86_64,317ecf7d1128e0e53cb285b8704dc3d3,bare,2017-11-29T04:08:45Z,qcow2,centos,/v2/images/4a2b369d-f3bc-4b73-b50a-cbd3687be43...,true,NaN,0.0,...,7,None,public,47af134d8877c9ba290af6853b3914ca5e385305359021...,NaN,NaN,NaN,NaN,NaN,NaN
160b5635-f30d-4d89-a70a-426872de22bc,x86_64,4b03de761dec3731c3d79d0b27240425,bare,2017-11-23T04:00:46Z,qcow2,ubuntu,/v2/images/160b5635-f30d-4d89-a70a-426872de22b...,true,32c0b2d60132455957b6e3a6767344992428a33c4500ae...,0.0,...,14.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN
f9035cc9-56cb-4336-9a2a-afe7824f7454,x86_64,b75012b0dcce71ff9be2036e7ae5bc92,bare,2017-11-23T04:00:26Z,qcow2,ubuntu,/v2/images/f9035cc9-56cb-4336-9a2a-afe7824f745...,true,c627f86e58f7e29ba15c3535f7763fd80628ee4c1ec195...,0.0,...,16.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN
cc66bcf5-a9d9-4a67-b9ca-3181a4bff74e,x86_64,c3e0b581d613a4806e6e5fd823d93f01,bare,2017-11-22T04:00:38Z,qcow2,ubuntu,/v2/images/cc66bcf5-a9d9-4a67-b9ca-3181a4bff74...,true,a9dd6590b374fb3c600f98b2b0ea8708ddbde5a6180d77...,0.0,...,16.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN
bb7c1380-2d68-4bdd-bffb-8fe7f317bbb0,x86_64,99e73c2c09cad6a681b2d372c37f2e11,bare,2017-11-19T04:00:25Z,qcow2,ubuntu,/v2/images/bb7c1380-2d68-4bdd-bffb-8fe7f317bbb...,true,ac850bef8dc9556c08fd57b9ed2321f23c67ea1a2016a9...,0.0,...,16.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5c46843e-3a75-49f8-b2c9-d52ad78449a6,x86_64,28c7994434128234d8b3b9807ef7a1a8,bare,2017-11-17T04:00:27Z,qcow2,ubuntu,/v2/images/5c46843e-3a75-49f8-b2c9-d52ad78449a...,true,800a1b72b05d3ad3dfabc60fd9e853ed1c8bf5a21a4e5d...,0.0,...,16.04,None,public,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Podemos filtrar las imágenes pasándole un filtro. Vamos a quedarnos con las imágenes que sean Ubuntu 16.04

In [17]:
filters = {"distribution": "ubuntu", "version": "16.04"}
images = glance.images.list(filters=filters)

In [18]:
to_dataframe(images)

,architecture,checksum,container_format,created_at,disk_format,distribution,file,ifca,imgsync.sha256,min_disk,...,size,source,status,tags,type,updated_at,version,virtual_size,visibility,os_command_line
id,,,,,,,,,,,,,,,,,,,,,
288d0d6a-17e2-4f16-a230-aa288dd146e1,x86_64,921539f1ebcaf8e822e777444e6f5090,bare,2017-11-30T04:00:27Z,qcow2,ubuntu,/v2/images/288d0d6a-17e2-4f16-a230-aa288dd146e...,true,0c6ac917eb58c524d72d30d694f7b21ea6e09b696fbb72...,0.0,...,2.932081e+08,imgsync,active,[],Linux,2017-11-30T04:00:30Z,16.04,None,public,NaN
f9035cc9-56cb-4336-9a2a-afe7824f7454,x86_64,b75012b0dcce71ff9be2036e7ae5bc92,bare,2017-11-23T04:00:26Z,qcow2,ubuntu,/v2/images/f9035cc9-56cb-4336-9a2a-afe7824f745...,true,c627f86e58f7e29ba15c3535f7763fd80628ee4c1ec195...,0.0,...,2.932736e+08,imgsync,active,[],Linux,2017-11-23T04:00:29Z,16.04,None,public,NaN
cc66bcf5-a9d9-4a67-b9ca-3181a4bff74e,x86_64,c3e0b581d613a4806e6e5fd823d93f01,bare,2017-11-22T04:00:38Z,qcow2,ubuntu,/v2/images/cc66bcf5-a9d9-4a67-b9ca-3181a4bff74...,true,a9dd6590b374fb3c600f98b2b0ea8708ddbde5a6180d77...,0.0,...,2.932736e+08,imgsync,active,[],Linux,2017-11-22T04:00:41Z,16.04,None,public,NaN
bb7c1380-2d68-4bdd-bffb-8fe7f317bbb0,x86_64,99e73c2c09cad6a681b2d372c37f2e11,bare,2017-11-19T04:00:25Z,qcow2,ubuntu,/v2/images/bb7c1380-2d68-4bdd-bffb-8fe7f317bbb...,true,ac850bef8dc9556c08fd57b9ed2321f23c67ea1a2016a9...,0.0,...,2.934047e+08,imgsync,active,[],Linux,2017-11-19T04:00:29Z,16.04,None,public,NaN
5c46843e-3a75-49f8-b2c9-d52ad78449a6,x86_64,28c7994434128234d8b3b9807ef7a1a8,bare,2017-11-17T04:00:27Z,qcow2,ubuntu,/v2/images/5c46843e-3a75-49f8-b2c9-d52ad78449a...,true,800a1b72b05d3ad3dfabc60fd9e853ed1c8bf5a21a4e5d...,0.0,...,2.932081e+08,imgsync,active,[],Linux,2017-11-17T04:00:30Z,16.04,None,public,NaN
f58fb90f-789b-45a6-8bfd-2371d56a2d06,x86_64,f81948dd96d14de49e1fee9208bbbcda,bare,2017-11-16T04:00:32Z,qcow2,ubuntu,/v2/images/f58fb90f-789b-45a6-8bfd-2371d56a2d0...,true,7034e4ab6887d2d6ba1c3603f5d403c5782abd6b8595b9...,0.0,...,2.931425e+08,imgsync,active,[],Linux,2017-11-16T04:00:36Z,16.04,None,public,NaN
6bf6f27d-1116-4fdd-97d7-19bdf6cf2b80,x86_64,41f023431eab19ce01eb73daa9d75809,bare,2017-11-11T04:00:30Z,qcow2,ubuntu,/v2/images/6bf6f27d-1116-4fdd-97d7-19bdf6cf2b8...,true,39624966d52a8f49104b4f6d4539dc4607c31bf16f8a17...,0.0,...,2.931425e+08,imgsync,active,[],Linux,2017-11-11T04:00:33Z,16.04,None,public,NaN
795ce42c-c05e-4e04-b24a-4e19e4e68b83,x86_64,e4f0ecf139ec4ea7b7403ed3cd06c382,bare,2017-11-10T04:00:26Z,qcow2,ubuntu,/v2/images/795ce42c-c05e-4e04-b24a-4e19e4e68b8...,true,86716e89d9cccdb4707b92f58957a4c0d1c2b2b08bf7a2...,0.0,...,2.933391e+08,imgsync,active,[],Linux,2017-11-10T04:00:29Z,16.04,None,public,NaN
1cf648a9-7de0-4106-abd0-71ecf9ac3ef5,x86_64,7e24f437418013d6600ac9f998a0e617,bare,2017-11-09T04:00:27Z,qcow2,ubuntu,/v2/images/1cf648a9-7de0-4106-abd0-71ecf9ac3ef...,true,df920257937c753d5adc8e7ba5b851dd97e7761a608383...,0.0,...,2.907177e+08,imgsync,active,[],Linux,2017-11-09T04:00:30Z,16.04,None,public,NaN


Vamos a utilizar la imagen `288d0d6a-17e2-4f16-a230-aa288dd146e1`

In [19]:
IMAGE_ID = "288d0d6a-17e2-4f16-a230-aa288dd146e1"

Normalmente no es necesario hacer todos los pasos anteriores ya que deberíamos saber con anterioridad que imagen y flavor vamos a necesitar, pero es un buen ejemplo de como interactuar con el servicio a través de la API.

# Creación de la máquina
Ya tenemos la imagen (`IMAGE_ID`) y el flavor (`FLAVOR_ID`) que queremos utilizar, así que ya estamos listos para arrancar la máquina.

Pero antes, tenemos que seleccionar que keypair vamos a utiliar

In [20]:
nova.keypairs.list()

[<Keypair: test>]

In [21]:
mi_server = nova.servers.create(name="test %s" % USERNAME,
                                image=IMAGE_ID,
                                flavor=FLAVOR_ID,
                                keypair="test")
mi_server

<Server: test master>

In [22]:
mi_server.status

'BUILD'

In [23]:
print("Server ID:", mi_server.id)
print("Server name:", mi_server.name)
print("Server image:", mi_server.image)

Server ID: 9bceab52-955e-4935-863c-a58c28d1cba4
Server name: test master
Server image: {'id': '288d0d6a-17e2-4f16-a230-aa288dd146e1', 'links': [{'href': 'https://cloud.ifca.es:8774/d2828a5b2eda4ad6a6de5cf3ef41df64/images/288d0d6a-17e2-4f16-a230-aa288dd146e1', 'rel': 'bookmark'}]}


In [24]:
mi_server = nova.servers.get(mi_server.id)

In [25]:
mi_server.status

'ACTIVE'

# Acceso a la máquina
Normalmente para acceder a la máquina se utilizaría Secure Shell (SSH), utilizando un par de llaves (privada-pública), para ello habría que:

1. Crear un par de llaves en Nova, utilizando `nova.keypairs.create` (no hay que crear un par de claves por cada máquina, sino que podemos crear tan solo uno). 
1. Asociar este par de claves en el momento de la creación (`nova.servers.create`), con el parámetro `keypair`.
1. Conectar a la nueva máquina, utilizando la llave privada que nos hemos bajado anteriormente.

Las instancias creadas en el cloud del IFCA están creadas en una red privada y solo se ven las máquinas del mismo proyecto. Para poder acceder desde el exterior hace falta utilizar una IP pública.

## Creación de una IP pública (flotante)

De nuevo, necesitamos interactuar con el componente de red, neutron, para pedirle una IP flotante (pública)

In [26]:
neutron = net_client.Client(session=sess, insecure=True)

In [27]:
neutron.list_floatingips()

{'floatingips': []}

In [28]:
# Tenemos que indicarle que es una red externa, en el caso del IFCA
# esta es "cfdf9350-ab32-40db-ad1f-9790303d79dd"

req = {'floatingip':{'floating_network_id': "cfdf9350-ab32-40db-ad1f-9790303d79dd"}}

ip = neutron.create_floatingip(req)

In [29]:
ip["floatingip"]["floating_ip_address"]

'193.146.75.243'

In [30]:
mi_server.add_floating_ip(ip["floatingip"]["floating_ip_address"])

()

Ahora podemos conectaar a través de SSH, a la interfaz que hemos asociado:

    ssh -i test.pem ubuntu@<IP>

In [31]:
# Tambien podemos ver la consola
print(mi_server.get_console_output())

[    0.000000] Initializing cgroup subsys cpuset
[    0.000000] Initializing cgroup subsys cpu
[    0.000000] Initializing cgroup subsys cpuacct
[    0.000000] Linux version 4.4.0-101-generic (buildd@lcy01-amd64-006) (gcc version 5.4.0 20160609 (Ubuntu 5.4.0-6ubuntu1~16.04.5) ) #124-Ubuntu SMP Fri Nov 10 18:29:59 UTC 2017 (Ubuntu 4.4.0-101.124-generic 4.4.95)
[    0.000000] Command line: root=LABEL=cloudimg-rootfs ro  console=tty1 console=ttyS0 console=tty0 console=ttyS0 console=hvc0
[    0.000000] KERNEL supported cpus:
[    0.000000]   Intel GenuineIntel
[    0.000000]   AMD AuthenticAMD
[    0.000000]   Centaur CentaurHauls
[    0.000000] x86/fpu: xstate_offset[2]:  576, xstate_sizes[2]:  256
[    0.000000] x86/fpu: Supporting XSAVE feature 0x01: 'x87 floating point registers'
[    0.000000] x86/fpu: Supporting XSAVE feature 0x02: 'SSE registers'
[    0.000000] x86/fpu: Supporting XSAVE feature 0x04: 'AVX registers'
[    0.000000] x86/fpu: Enabled xstate features 0x7, context size i

In [32]:
# O conectarnos a ella
mi_server.get_console_url(console_type="novnc")

{'console': {'type': 'novnc',
  'url': 'https://cloud.ifca.es:6080/vnc_auto.html?token=f0825288-afd5-4bde-8f57-0974d58f43ce'}}